## Lectura de archivos

In [ ]:
# Versión 1
f = open('./sample_data/california_housing_test.csv', 'r')
contenido = ''
x = f.readline()
while x != '':
  contenido =  contenido + x 
  x = f.readline()
f.close()
# Fin versión 1

# Está mal 
f = open('./sample_data/california_housing_test.csv', 'r')
contenido = ''

while f.readline() != '':
  contenido =  contenido + f.readline() 
f.close()
# fin Está mal 


# Versión 2
f = open('./sample_data/anscombe.json', 'r')
contenido = ''
for linea in f:
  contenido = contenido + linea
f.close()
# Fin versión 2

# Versión 3
with open('./sample_data/anscombe.json') as f:
  contenido = f.read()
# Fin versión 3



**Introducción** 

La Oficina Nacional de Administración Oceánica y Atmosférica (National Oceanic and Atmospheric Administration, NOAA) es una agencia científica del Departamento de Comercio de los Estados Unidos cuyas actividades se centran en monitorear las condiciones de los océanos y la atmósfera. 

El NOAA ofrece distintos *datasets* (conjuntos de datos estructurados) sin limitaciones. Los datos puedes ser bajados directamente del sitio web https://www.noaa.gov/ o, alternativamente, accedidos por API. 

En este ejercicio accederemos a los datasets utilizando la API. Para utilizar la API se debe solicitar  un *token* al NOAA (ver *Referencias*). El token es una cadena de caracteres que funciona como id y password al mismo tiempo. 

Si al comienzo ponemos 

    import requests
    token = 'nuestro token'
    my_headers = {'token' : token}
    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', headers=my_headers)
    respuesta = response.json()
    resultados = respuesta['results'] # Datasets disponible
    for w in resultados:
      pass
      print(w['uid'],':',w['name'])

obtendremos el `id` de cada uno de los datasets ofrecidos por el NOAA via API y una breve descripción de los mismos. En particular,  son interesantes:

    GHCND : Daily Summaries
    GSOM : Global Summary of the Month
    GSOY : Global Summary of the Year

La API permite una variedad de consultas, por ejemplo

    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?locationid=CITY:US390029', headers=my_headers)
    respuesta = response.json()

nos devuelve todos los datasets disponibles para determinada localidad. 

También podemos obtener, por ejemplo, datos climatológicos de una determinada localidad durante un período determinado de tiempo: la consulta

    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:AR000087344&units=metric&startdate=2021-05-01&enddate=2021-05-31', headers=my_headers)
    respuesta = response.json()
    resultados = respuesta['results']

nos devuelve un sumario de temperaturas y precipitaciones diarias en el mes de mayo de la estación meteorológica del Aeropuerto de Córdoba.  

En las referencias a continuación se encuentra como obtener el token, la descripción de la API, la lista de estaciones meteorológicas y mucha más información. Luego viene una celda de código con ejemplos, y luego una celda de texto con el enunciado del ejercicio. 



*Referencias*
- https://www.ncdc.noaa.gov/ National Centers for Environmental Information (37 petabytes)
- https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation
- Para bajar estaciones meteorolgógicas:  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
- https://www.ncdc.noaa.gov/ghcn-daily-description
- Para bajar manual de ghcn-daily: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
- Para pedir token: https://www.ncdc.noaa.gov/cdo-web/token
- Climate Data Online - Web Services Documentation: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted
- Manual genérico para uso de APIs en Python: https://www.nylas.com/blog/use-python-requests-module-rest-apis/




In [ ]:
import requests
token = 'LsJXpYprYMzOKjMtqAlDmMUzwGelIQDf'
my_headers = {'token' : token}
# Ejemplo: datasets disponibles en NOAA
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', headers=my_headers)
respuesta = response.json()

resultados = respuesta['results'] # Datasets disponibles
"""
for w in resultados:
  pass
  print(w['id'],':',w['name'])
"""

# Ejemplo: datasets  disponible en la ciudad US390029
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?locationid=CITY:US390029', headers=my_headers)
respuesta = response.json()
# print(respuesta)

# Datos climatológicos diarios (temperatura máxima, temperatura mínima y precipitaciones) en un período de tiempo en
# la estación meteorológica  AR000087344 (Aeropuerto de Córdoba) 
fecha_ini, fecha_fin = '2010-05-01', '2010-05-31'
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:AR000087344&limit=1000&units=metric&startdate='+fecha_ini+'&enddate='+fecha_fin+'', headers=my_headers)
respuesta = response.json()
resultados = respuesta['results']
# print(response.json())

for w in resultados:
  print(w)
# Observación 1. Cuidado: no necesariamente todos los día tienen todos los datos.
# Observación 2. Por defecto la API  recupera 25 registros por llamada. En la llamada anterior pusimos 'limit=1000'
#                para recuperar hasta 1000 registros por llamada. La API no permite más. 

print('')

Observar que cada elemento de  `resultados` es un diccionario y en ese diccionario  estan las claves`date`, `datatype` y `value`. 
- `date` indica el día. 
- Si el `datatype` es `TMAX`,  entonces `value` indica la temperatura máxima del día.
- Si el `datatype` es `TMIN`,  entonces `value` indica la temperatura mínima del día.
- Si el `datatype` es `TAVG`,  entonces `value` indica la temperatura promedio del día.
- Si el `datatype` es `PRCP`,  entonces `value` indica la precipitación (en mm) de ese día. 

## Lectura y escritura de json

a) Es importante recalcar que

        json.dumps(`tipo diccionario o lista`) -> `tipo str`
        json.loads(`tipo str`) -> `tipo diccionario o lista`
Así debe usarse. Cualquier otro tipo de uso quizás Python lo permita pero puede traes problemas. Por otro lado,  hay que tener en cuenta que solo a strings muy especiales se puede aplicar `json.loads()`

b) Todo lo que va a ser leido por `json.loads()` o viene de otra fuente (bien formateada)  o debe haber sido creado por `json.dumps()`.

c) Después de un `json.dumps()` todas las claves quedan entre comillas dobles, luego si se carga de nuevo el diccionario con `json.loads()` las claves numéricas pasan a ser claves de tipo cadena. Los valores se mantienen en el tipo original. Por ejemplo:

        x = json.dumps({'a': {2 : 0 }, 'b': {1 : 5 }}) # x es la cadena '{"a": {"2" : 0 }, "b": {"1" : 5 }}'
        y = json.loads(x) # y  es el diccionario {'a': {'2' : 0 }, 'b': {'1' : 5 }}




In [ ]:
import json 
u = {'a': {2 : 0 }, 'b': {1 : 5 }}
print (type(u), u)
x = json.dumps({'a': {2 : 0 }, 'b': {1 : 5 }})
print(type(x), x)
y = json.loads(x)
print(type(y), y)

# observar que el diccionario original cambia las claves numérica de TODOS los diccionarios a tipo str, incluso los diccionarios que son valores de otro diccionario.
# Los valores numéricos se mantienen numéricos. Si un valor es un lista,  se mantiene lista.

<class 'dict'> {'a': {2: 0}, 'b': {1: 5}}
<class 'str'> {"a": {"2": 0}, "b": {"1": 5}}
<class 'dict'> {'a': {'2': 0}, 'b': {'1': 5}}


*Ejemplo* Tenemos una lista de diccionarios y queremos escribir uno por línea en un archivo, para que luego sea leido usando `json.loads()`. En  la celda de código siguiente hacemos un ejemplo. 

In [ ]:
import json 
import random

# Creamos una lista con 100 diccionarios tipo { 'x' : entero al azar, 'y' : entero al azar, nº de orden en la lista : [entero al azar, entero al azar] }
# Los guardaremos en el archivo prueba.txt, uno en cada linea. 

lista_dic = []
for i in range(100):
  x, y, w0, w1 = random.randint(1, 1000), random.randint(1, 1000), random.randint(1, 1000), random.randint(1, 1000)
  lista_dic.append({ 'x' : x, 'y' : y, i : [w0 , w1]})

# Guardamos en el archivo prueba.txt, línea por línea el diccionario.
f = open('prueba.txt', 'w')
for i in range(len(lista_dic)):
  f.write(json.dumps(lista_dic[i])+'\n') # \n pasa de renglón
f.close()

# Revisar el archivo prueba.txt

# Para leer debemos hacerlo también linea por linea, pues el archivo no tiene la estructura lógica de un diccionario o lista 
# (debería abrir con { o [ y cerrar con } o  ],  respectivamente)

# Ahora haremos una lista de diccionarios a partir de prueba.txt

lista_dic2 = []
f = open('prueba.txt')
for linea in f:
  lista_dic2.append(json.loads(linea))
f.close()

print(type(lista_dic2[3]), lista_dic2[3]) # comprobamos que cada coordenada es un diccionario

# Importante:  las claves numéricas pasaron a str. Los valores matienen su tipo (int, list, etc)
print(lista_dic[3][3]) 
print(lista_dic2[3]['3']) # No estaría bien  print(lista_dic2[3][3]) 


*Ejemplos de cosas que  **NO** funcionan.*


In [ ]:
import json
# Si hacemos json.dumps()  de una string, Python lo hace, pero no es el resultado que esperamos.
x = json.dumps("{'a': {2 : 0 }, 'b': {1 : 5 }")
print(type(x), x)
y = json.loads(x)
print(type(y), y) # json.loads() en este caso devuelve una string!
# y es una string que "parece" un diccionario. Pero no lo es. 
# z = json.loads(y) #  esto directamente da error


<class 'str'> "{'a': {2 : 0 }, 'b': {1 : 5 }"
<class 'str'> {'a': {2 : 0 }, 'b': {1 : 5 }


**Recomendación:** Usar `json.loads()` y `json.dumps()` de forma prolija y de acuerdo a los lineamientos escritos más arriba. Quizás haciendo aguna  "magia" con los `replace()` se pueda arreglar la celda de código anterior, pero es mejor no hacerlo.

## Criba de Eratóstenes (iterativa)

En  la siguiente celda de código se da una versión iterativa de la criba de Eratóstenes. 

In [ ]:
def criba(n):
  # pre: n número natural
  # post: se obtiene ''primos'' la lista de números primos hasta n
  primos = [] # lista vacía
  tachados = [] # lista de números tachados 
  for i in range(2, n + 1):
      if i not in tachados:
          primos.append(i) # agregar i a primos
          k = 2
          while k * i <= n:
              tachados.append(k * i) # agrega k*i a tachados
              k = k + 1
  return primos

print('Primos: ', criba(150))

# El algoritmo de arriba es muy ineficiente. En particular i recorre 
# todos los números hasta n y hace comprobaciones innecesarias.

# Un algoritmo optimizado es el siguiente (es el de Wikipedia en inglés)

def criba_w(n): # de Wikipedia en inglés.
    a = [True]*(n+1) # Hace un  lista de n+1 elementos cada uno True
    for i in range(2, int(n**0.5) + 1):
        if a[i] == True:
            for j in range(i**2, n+1, i ):
                a[j] = False
    # Si a[i] == True,  entonces i es primo (i >= 2)
    return [i for i in range(2,n+1) if a[i] == True]

# La diferencia de tiempos entre el primer el algoritmo y el segundo es significativa.  

Primos:  [2, 3, 5, 7, 11, 13]


In [1]:
# Python permite también hacerlo con conjuntos por comprensión. 
# Esta es una solución poco general (muy del lenguage Python)
# Tampoco es estrictamente la criba de Eratóstenes, pues elimina todos los  k * x para k >= 2 sin importar si x es primo. 


n = 150
a =  set(range(2, n + 1)) -  {k for i in range(2, int(n**0.5) + 1) for k in range(i**2, n + 1, i )}
print('Primos: ', sorted(list(a)))

# Pese a que hace pasos innecesarios (eliminar los k * x con x compuesto) es un algoritmo relativamente eficiente.


Primos:  [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149]
